# Unsloth를 이용한 Gemma Model Fine Tuning

Unsloth를 이용하면 메모리 Foot Print를 최소화하면서, PEFT를 효율적으로 진행할 수 있습니다. 

패키지 설치, 데이터셋 구성, 트레이닝 이후 테스트까지 진행해보도록 하겠습니다. 


In [ ]:
! python --version

이 셀에서는 필요한 Python 패키지를 설치합니다. `torch`, `torchvision`, `torchaudio`, `xformers`는 PyTorch 생태계의 핵심 라이브러리이며, 딥러닝 모델을 구축하고 학습하는 데 사용됩니다. 특히, `xformers`는 메모리 효율적이고 빠른 연산을 위한 최적화된 연산자를 제공합니다. `unsloth`는 대규모 언어 모델을 미세 조정하는 과정을 단순화하고 최적화하는 데 초점을 맞춘 라이브러리입니다. 이 라이브러리는 모델 로딩, 미세 조정, 추론 과정을 간편하게 만들어줍니다. 마지막으로, `transformers`는 Hugging Face에서 개발한 자연어 처리 모델 라이브러리입니다. 이 셀에서는 `transformers` 라이브러리를 특정 버전으로 강제 설치하여 호환성 문제를 방지하고 있습니다.

**주요 포인트:**

*   **PyTorch 생태계:** 딥러닝 모델 개발에 필수적인 라이브러리들을 설치합니다.
*   **`unsloth` 라이브러리:** 대규모 언어 모델의 미세 조정을 위한 핵심 라이브러리로, 설치함으로써 미세 조정 과정을 단순화합니다.
*   **`transformers` 라이브러리:** 특정 버전으로 고정하여 호환성 문제를 방지합니다. 이는 특정 버전의 라이브러리에 최적화된 코드를 사용하거나, 라이브러리 간의 충돌을 방지하기 위함입니다.
*   **GPU 가속:** CUDA 12.1 버전에 맞춰 PyTorch 및 관련 라이브러리를 설치하여 GPU 가속을 활용합니다. 이는 모델 학습 및 추론 속도를 향상시킵니다.

In [ ]:
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install 'unsloth==2025.1.1'
!pip uninstall transformers -y
!pip install 'transformers==4.47.1'

패키지가 재설치되었다면 반드시 RESTART를 수행해주세요. 

이 셀에서는 필요한 파이썬 라이브러리를 임포트하고, 미세 조정에 필요한 설정을 정의합니다. `unsloth` 라이브러리에서 `FastLanguageModel` 클래스를 임포트하여 모델 로딩 및 조작을 용이하게 합니다. `torch`는 PyTorch 라이브러리의 핵심 모듈이며, 텐서 연산 및 GPU 가속을 위해 사용됩니다. `max_seq_length` 변수는 모델이 처리할 수 있는 최대 시퀀스 길이를 설정하며, 여기서는 8192 토큰으로 설정되었습니다. `dtype` 변수는 데이터 타입을 정의하며, 여기서는 `None`으로 설정되어 있어 자동으로 감지되도록 합니다. `load_in_4bit` 변수는 모델 가중치를 4비트 정밀도로 로드할지 여부를 결정하며, 여기서는 `True`로 설정되어 메모리 사용량을 줄이고 모델 로딩 속도를 향상시킵니다.

**주요 포인트:**

*   **`FastLanguageModel` 클래스:** `unsloth` 라이브러리의 핵심 클래스로, 모델 로딩 및 조작을 간편하게 합니다.
*   **`torch` 라이브러리:** 텐서 연산, GPU 가속 등 딥러닝 모델링에 필수적인 기능을 제공합니다.
*   **최대 시퀀스 길이(`max_seq_length`):** 모델이 처리할 수 있는 최대 입력 토큰 수를 정의합니다. 모델이 긴 문맥을 처리할 수 있도록 설정합니다.
*   **데이터 타입(`dtype`):** 모델 가중치 및 연산에 사용될 데이터 타입을 설정합니다. `None`으로 설정하여 자동 감지를 활성화합니다.
*   **4비트 로딩(`load_in_4bit`):** 모델 가중치를 4비트 정밀도로 로드하여 메모리 사용량을 줄이고 모델 로딩 시간을 단축합니다.

In [ ]:
# Import the FastLanguageModel class from the unsloth library.
from unsloth import FastLanguageModel
# Import the torch library.
import torch

# Set the maximum sequence length to 8192 tokens.
max_seq_length = 8192

# Set the data type to None for automatic detection.
dtype = None

# Set the load_in_4bit flag to True to load the model weights in 4-bit precision.
load_in_4bit = True

이 셀에서는 `unsloth` 라이브러리를 사용하여 Gemma-7b 모델을 로드합니다. `FastLanguageModel.from_pretrained()` 함수를 사용하여 모델과 토크나이저를 로드합니다. `model_name` 매개변수는 로드할 모델의 이름(여기서는 "unsloth/gemma-7b-bnb-4bit")을 지정합니다. 이 모델은 4비트 정밀도로 양자화된 버전으로, 메모리 효율적으로 모델을 사용할 수 있도록 해줍니다. 앞서 설정한 `max_seq_length`, `dtype`, `load_in_4bit` 매개변수를 전달하여 모델 로딩을 사용자 정의합니다.

**주요 포인트:**

*   **`from_pretrained()` 함수:** `unsloth` 라이브러리에서 제공하는 함수로, 사전 학습된 모델과 토크나이저를 편리하게 로드합니다.
*   **Gemma-7b 모델:** Google에서 개발한 대규모 언어 모델 중 하나이며, 70억 개의 파라미터를 가지고 있습니다.
*   **4비트 양자화 모델:** 모델 가중치를 4비트 정밀도로 양자화하여 메모리 사용량을 줄이고 모델 로딩 시간을 단축합니다. 이는 제한된 GPU 메모리 환경에서 대규모 모델을 효과적으로 사용할 수 있도록 해줍니다.
*   **미세 조정 설정:** 미리 정의된 설정(최대 시퀀스 길이, 데이터 타입, 4비트 로딩)을 사용하여 모델을 로드합니다.

In [ ]:
# Load the pre-trained model from the 'unsloth/gemma-7b-bnb-4bit' repository.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit",
    #model_name = "google/gemma-7b",
    
    # Set the maximum sequence length to the value defined earlier.
    max_seq_length = max_seq_length,

    # Set the data type to the value defined earlier.
    dtype = dtype,

    # Set the load_in_4bit flag to the value defined earlier.
    load_in_4bit = load_in_4bit,
)

이 셀에서는 로드된 모델에 LoRA(Low-Rank Adaptation)를 적용하여 미세 조정 가능한 파라미터 수를 줄입니다. `FastLanguageModel.get_peft_model()` 함수를 사용하여 모델을 래핑하고 LoRA 설정을 적용합니다. `r` 매개변수는 LoRA의 랭크를 설정하며, 16으로 설정되었습니다. `target_modules` 매개변수는 LoRA를 적용할 레이어의 이름을 지정합니다. 여기서는 Transformer 레이어의 핵심 연산 레이어(q_proj, k_proj, v_proj, o_proj 및 MLP 레이어)에 적용하도록 설정되어 있습니다. `lora_alpha` 및 `lora_dropout` 매개변수는 LoRA의 학습률과 드롭아웃 비율을 조절합니다. `bias` 매개변수는 바이어스를 어떻게 처리할지 설정하며, `none`으로 설정하여 바이어스를 사용하지 않습니다. 마지막으로 `use_gradient_checkpointing`를 True로 설정하여 메모리 사용을 최적화합니다.

**주요 포인트:**

*   **LoRA(Low-Rank Adaptation):** 모델 전체를 업데이트하는 대신 저랭크 행렬을 학습하여 미세 조정 가능한 파라미터 수를 줄이는 기법입니다. 이는 대규모 모델의 미세 조정에 필요한 계산량과 메모리 사용량을 줄여줍니다.
*   **LoRA 랭크(`r`):** LoRA 행렬의 랭크를 설정합니다. 낮은 랭크를 사용할수록 모델의 파라미터 수는 줄어들지만, 모델의 표현력에 영향을 줄 수 있습니다. 여기서는 16으로 설정하여 적절한 균형을 유지합니다.
*   **타겟 모듈(`target_modules`):** LoRA를 적용할 특정 레이어(Transformer 레이어의 핵심 연산 레이어)를 지정합니다.
*   **기타 파라미터:** `lora_alpha`, `lora_dropout`, `bias` 등의 파라미터를 통해 LoRA 학습 과정을 조절합니다.
*   **Gradient Checkpointing:** 메모리 사용량을 줄이면서 학습을 수행하는 기술입니다. 큰 모델을 학습할 때 매우 유용합니다.

In [ ]:
# Create a PEFT model with the given parameters
model = FastLanguageModel.get_peft_model(
    model,
    r=16, # LoRa Rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True
)

이 셀에서는 학습 데이터를 특정 형식으로 변환하는 함수 `formatted_train`를 정의합니다. 이 함수는 입력 데이터의 구조를 확인하고, 'instruction', 'input', 'output' 필드를 사용하여 텍스트를 특정 형식으로 결합합니다. `eos` 토큰을 추가하여 각 응답이 끝나는 부분을 명확하게 표시합니다. 입력 데이터에 'input' 필드가 있으면, 해당 내용을 포함한 형식으로 만들고, 없으면 'input' 필드 없이 'instruction'과 'output'만 사용해서 텍스트를 만듭니다.

**주요 포인트:**

*   **학습 데이터 형식:** 학습 데이터를 모델에 입력하기 전에 특정 형식으로 포맷팅하여 모델이 더 효과적으로 학습할 수 있도록 준비합니다.
*   **`instruction`, `input`, `output`:** 데이터셋의 각 항목은 'instruction'(지시), 'input'(입력), 'output'(출력) 세 가지 필드를 포함합니다. 이들을 사용하여 모델 학습에 적합한 텍스트를 생성합니다.
*   **`eos` 토큰:** 문장의 끝을 나타내는 특수 토큰을 추가하여 모델이 응답의 끝을 명확하게 인식할 수 있도록 합니다. 이는 모델이 불필요하게 응답을 생성하는 것을 방지합니다.
*   **조건부 형식:** 입력 데이터에 'input' 필드가 있는지 여부에 따라 다른 형식으로 출력합니다.

In [4]:
def formatted_train(x):

  if x['input']:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Input:
    {x['input']}

    ### Response:
    {x['output']}<eos>"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Response:
    {x['output']}<eos>"""

  return formatted_text

이 셀에서는 Hugging Face `datasets` 라이브러리를 사용하여 학습 데이터를 로드하고 전처리합니다. `load_dataset()` 함수를 사용하여 지정된 데이터셋("TokenBender/code_instructions_122k_alpaca_style")을 로드합니다. 로드된 데이터셋을 pandas DataFrame으로 변환하고, 앞서 정의한 `formatted_train` 함수를 사용하여 각 데이터 항목을 포맷팅합니다. 마지막으로, 포맷팅된 데이터를 다시 `datasets.Dataset` 객체로 변환합니다.

**주요 포인트:**

*   **Hugging Face `datasets` 라이브러리:** 대규모 데이터셋을 쉽게 로드하고 관리할 수 있는 라이브러리입니다.
*   **데이터셋 로드:** `load_dataset()` 함수를 사용하여 학습 데이터셋을 로드합니다.
*   **DataFrame 변환:** 데이터셋을 pandas DataFrame으로 변환하여 데이터 조작을 용이하게 합니다.
*   **데이터 포맷팅:** `formatted_train` 함수를 사용하여 데이터셋의 각 항목을 특정 형식으로 변환합니다.
*   **Dataset 객체 변환:** 포맷팅된 데이터를 다시 `datasets.Dataset` 객체로 변환하여 모델 학습에 사용할 수 있도록 합니다.


In [ ]:
from datasets import load_dataset, Dataset

def prepare_train_data(data_id):
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()
    data_df["formatted_text"] = data_df[["input", "output",
    "instruction"]].apply(formatted_train, axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data_id = "TokenBender/code_instructions_122k_alpaca_style"

data = prepare_train_data(data_id)

이 셀에서는 `trl` 라이브러리의 `SFTTrainer` 클래스를 사용하여 모델을 미세 조정합니다. `SFTTrainer`는 Supervised Fine-Tuning(지도 미세 조정)을 위한 트레이너입니다. 모델, 토크나이저, 학습 데이터셋, 데이터셋의 텍스트 필드, 최대 시퀀스 길이 등의 매개변수를 설정합니다. `TrainingArguments` 객체를 사용하여 학습률, 배치 크기, 학습 스텝, 옵티마이저, 로깅 빈도 등 학습과 관련된 하이퍼파라미터를 설정합니다. 특히, `fp16`와 `bf16`을 사용하여 학습 시에 메모리 사용량을 줄이고 학습 속도를 향상시킵니다. `paged_adamw_8bit` 옵티마이저는 메모리 사용량을 줄이는 동시에 학습 성능을 높이는 데 사용됩니다.

**주요 포인트:**

*   **`SFTTrainer` 클래스:** Supervised Fine-Tuning을 위한 트레이너 클래스입니다. 모델을 사용자 정의된 데이터셋으로 학습시키는 데 필요한 모든 기능을 제공합니다.
*   **학습 데이터셋 설정:** 모델 학습에 사용될 데이터셋을 설정합니다.
*   **최대 시퀀스 길이:** 모델이 처리할 수 있는 최대 시퀀스 길이를 설정합니다.
*   **`TrainingArguments` 객체:** 학습에 필요한 하이퍼파라미터를 설정합니다. 학습률, 배치 크기, 학습 스텝 등을 설정하여 학습 프로세스를 제어합니다.
*   **`fp16` 및 `bf16` 혼합 정밀도 학습:** 학습 속도를 향상시키고 메모리 사용량을 줄입니다.
*   **`paged_adamw_8bit` 옵티마이저:** 메모리 효율적인 옵티마이저를 사용하여 학습 중에 메모리 문제를 줄입니다.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = data,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

이 셀에서는 `trainer.train()` 함수를 호출하여 모델 미세 조정을 시작합니다. `trainer.train()` 함수는 지정된 학습 데이터셋과 하이퍼파라미터를 사용하여 모델을 학습시키고, 학습 과정 동안의 손실과 같은 지표들을 로깅합니다.

**주요 포인트:**

*   **미세 조정 시작:** `trainer.train()` 함수를 호출하여 모델 미세 조정을 시작합니다.
*   **학습 과정 로깅:** 학습 과정 동안의 손실 및 기타 지표들을 로깅하여 학습 진행 상황을 모니터링합니다.
*   **Weights & Biases 연동:** 학습 과정의 시각화를 위해 Weights & Biases와 연동합니다.

In [ ]:
trainer_stats = trainer.train()

이 셀에서는 테스트 데이터를 생성하기 위해 `format_test` 함수를 정의합니다. 이 함수는 입력 데이터에 'input' 필드가 있는지 확인하고, 'instruction'과 'input'(있는 경우) 필드를 사용하여 테스트 프롬프트를 구성합니다. 학습과 마찬가지로, 입력 데이터에 'input'이 있는 경우와 없는 경우에 따라 서로 다른 형식으로 텍스트를 구성합니다.

**주요 포인트:**

*   **테스트 프롬프트 형식:** 테스트 프롬프트를 특정 형식으로 생성합니다.
*   **`instruction` 및 `input`:** 테스트 프롬프트는 'instruction'과 'input' 필드를 사용합니다.
*   **조건부 형식:** 입력 데이터에 'input' 필드가 있는지 여부에 따라 다른 형식으로 출력합니다.

In [8]:
def format_test(x):

  if x['input']:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Input:
    {x['input']}

    ### Response:
    """

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {x['instruction']}

    ### Response:
    """

  return formatted_text

이 셀에서는 학습된 모델을 사용하여 추론을 수행합니다. `format_test` 함수를 사용하여 테스트 프롬프트를 생성하고, `FastLanguageModel.for_inference()`를 사용하여 모델을 추론 모드로 전환합니다. 추론에 최적화된 커널을 활성화하여 추론 속도를 높입니다. 그런 다음, `tokenizer`를 사용하여 프롬프트를 토큰화하고, 생성된 토큰을 GPU로 이동시킵니다. `TextStreamer`를 사용하여 모델이 생성하는 텍스트를 실시간으로 스트리밍합니다. 마지막으로, `model.generate()` 함수를 사용하여 텍스트를 생성하고, `max_new_tokens`를 설정하여 생성되는 최대 토큰 수를 제한합니다.

**주요 포인트:**

*   **테스트 프롬프트 생성:** `format_test` 함수를 사용하여 테스트 프롬프트를 생성합니다.
*   **추론 모드 활성화:** `FastLanguageModel.for_inference()` 함수를 사용하여 모델을 추론 모드로 전환하고 추론 속도를 높입니다.
*   **프롬프트 토큰화:** `tokenizer`를 사용하여 테스트 프롬프트를 토큰화하고, 텐서 형태로 변환합니다.
*   **GPU 가속:** 토큰화된 텐서를 GPU로 이동시켜 추론 속도를 향상시킵니다.
*   **텍스트 스트리밍:** `TextStreamer`를 사용하여 모델이 생성하는 텍스트를 실시간으로 스트리밍합니다.
*   **텍스트 생성:** `model.generate()` 함수를 사용하여 텍스트를 생성하고, `max_new_tokens`를 사용하여 생성되는 최대 토큰 수를 제한합니다.

In [ ]:
Prompt = format_test(data[155])
print(Prompt)

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

오 모델이 정상적으로, 동작하는 것을 확인하였습니다.

Finetuned Model을 나중에라도 쓸수있게 저장하겠습니다.

In [ ]:
# 1. Define the local directory where you want to save the model.
local_model_path = "fine_tuned_model"

# 2. Save the model and tokenizer locally
trainer.save_model(local_model_path)
tokenizer.save_pretrained(local_model_path)
print(f"Model and tokenizer saved locally to {local_model_path}")


해당하는 모델을 GCS Bucket에 UPloading하게습니다.

사전에 Bucket이 있다면 해당 Bucket을 사용하고 없다면 아래에 있는 command를 이용해서 만들어 주세요.

In [ ]:
# prompt: make gcs bucket with given bucket name in gcloud command

BUCKET_NAME="vertexai-unsloth-yourname"
gcs_model_path = "fine_tuned_gemma_model" # The directory inside the bucket where the model will be uploaded

# uncomment the below line if you want to make a bucket.
# !gsutil rb gs://$BUCKET_NAME
# !gsutil mb -l asia-northeast3 gs://$BUCKET_NAME

In [ ]:
import os
from google.cloud import storage



# 4. Authenticate with Google Cloud Storage
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)


In [ ]:
def upload_directory_to_gcs(local_path, bucket, gcs_path):
    """Uploads all files in a local directory to GCS, preserving the directory structure."""
    for root, _, files in os.walk(local_path):
        for file in files:
            local_file = os.path.join(root, file)
            relative_path = os.path.relpath(local_file, local_path)
            gcs_file_path = os.path.join(gcs_path, relative_path).replace("\\", "/")  # For windows compatibility
            blob = bucket.blob(gcs_file_path)
            blob.upload_from_filename(local_file)
            print(f"Uploaded {local_file} to gs://{bucket.name}/{gcs_file_path}")

upload_directory_to_gcs(local_model_path, bucket, gcs_model_path)

print(f"\nModel successfully uploaded to gs://{BUCKET_NAME}/{gcs_model_path}")


## Fine Tuned Model 생성 완료

로컬에 Fine Tuned Model을 저장하여고, GCS에 해당 모델을 업로드 하였다.


## Vertex AI에 Fine Tuned Model Deploy

기존에 있던 모델활용은 해당 Workspace(또는 Google Colab)환경에서 실제 모델을 읽고 Local에서 활용하기 위한 방법입니다.

타인 또는 외부용 서비스로 제공하기 위해서는 공식적인 Endpoint를 구성하여 Scalibility와 Resiliency를 보장해야 합니다.

이를 위하여, 현재 구성된 모델을 Model Repository에 등록합니다.

이후, 등록된 모델을 Vertex AI Endpoint에 배포하고, 이를 활요하여 Inference를 수행합니다.

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part, ChatSession
from vertexai.language_models import TextGenerationModel
from google.cloud import aiplatform
# from google.oauth2 import service_account

In [ ]:
# prompt: get PROJECT_ID using gcloud command and store this value into PROJECT_ID variable in python

PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
PROJECT_ID

In [ ]:
BUCKET_NAME="vertexai-unsloth-yourname"
gcs_model_path = "fine_tuned_gemma_model" # The directory inside the bucket where the model will be uploaded

In [ ]:

REGION = "asia-northeast3" # Replace with your region
GCS_MODEL_PATH = f"gs://{BUCKET_NAME}/{gcs_model_path}" # Replace with your GCS path
ENDPOINT_NAME = "fine-tuned-gemma-endpoint" # Replace with your endpoint name
DEPLOYED_MODEL_NAME = "fine-tuned-gemma-model" # Replace with your model name
MODEL_DISPLAY_NAME = "fine-tuned-gemma"

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
def deploy_model_to_vertex_ai():

    model_upload = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=GCS_MODEL_PATH,
        description="Fine-tuned Gemma model",
        is_regionalized=True,
    )

    # Model Deploy
    endpoint = aiplatform.Endpoint.create(
        display_name=ENDPOINT_NAME,
        description="Endpoint for fine-tuned Gemma model",
    )

    deployed_model = endpoint.deploy(
        model=model_upload,
        deployed_model_display_name=DEPLOYED_MODEL_NAME,
        machine_type="g2-standard-4",  # g2 인스턴스 사용 (L4 GPU 포함)
        accelerator_type="NVIDIA_L4",  # L4 GPU 사용 명시
        accelerator_count=1, # GPU 개수 설정
    )

    print(f"Model deployed to endpoint: {endpoint.resource_name}")
    print(f"Deployed Model Resource Name: {deployed_model.resource_name}")

    return deployed_model.id, endpoint.id


In [ ]:
def predict_with_vertex_ai(deployed_model_id, endpoint_id, prompt):
    """
    Sends a prompt to a deployed Vertex AI model and returns the response.
    """

    # Initialize Vertex AI prediction client
    model = aiplatform.DeployedModel(
        endpoint=endpoint_id,
        deployed_model=deployed_model_id,
    )

    # Prepare Prediction input parameter
    instances = [
        {
            "prompt": prompt,
        }
    ]
    parameters = {
        "temperature": 0.2,
        "max_output_tokens": 512,
    }


    # Get predictions
    prediction = model.predict(instances=instances, parameters=parameters)

    # Extract response
    response = prediction.predictions[0]["content"]

    return response


In [ ]:
# 배포 및 모델 ID 가져오기
deployed_model_id, endpoint_id = deploy_model_to_vertex_ai()

# 추론할 프롬프트 설정
test_prompt = """Below is an instruction that describes a task.
Write a response that appropriately completes the request.
### Instruction:
Create a way to encrypt a message using a key in Python.
### Input:
message = "Hello world!"
key = "secret"
### Response:
"""

# 추론 실행
response = predict_with_vertex_ai(deployed_model_id, endpoint_id, test_prompt)
print("\n----- Prediction Result -----")
print(response)
